## Imports

In [1]:
import bilby
import gwpopulation as gwpop
import numpy as np
import matplotlib.pyplot as plt
import h5py
import pandas as pd
import corner
import seaborn as sns

from astropy import cosmology, units
from scipy.interpolate import interp1d
from scipy.special import erf

from bilby.core.prior import Uniform, LogUniform, PriorDict, Constraint
from gwpopulation.models import mass, spin

%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Model B - Gaussian

This paper uses `Overall_posterior`.

In [2]:
parameter_translator = dict(
    mass_1_det = 'm1_detector_frame_Msun',
    mass_2_det = 'm2_detector_frame_Msun',
    luminosity_distance = 'luminosity_distance_Mpc',
    a_1 = 'spin1',
    a_2 = 'spin2',
    cos_tilt_1 = 'costilt1',
    cos_tilt_2 = 'costilt2')

In [3]:
file = 'GWTC-1_sample_release/GW{}_GWTC-1.hdf5'

events = ['150914', '151012', '151226', '170104', '170608', '170729', '170809', '170814', 
          '170818', '170823']

posteriorsB = []
priorsB = []
event_name = []

for event in events:
    _posterior = pd.DataFrame()
    _prior = pd.DataFrame()
    with h5py.File(file.format(event), 'r') as ff:
        for my_key, gwtc_key in parameter_translator.items():
            _posterior[my_key] = ff['Overall_posterior'][gwtc_key]
    posteriorsB.append(_posterior)
    event_name.append(event)

In [4]:
luminosity_distances = np.linspace(1, 10000, 1000)
redshifts = np.array([cosmology.z_at_value(cosmology.Planck15.luminosity_distance, 
                                                   dl * units.Mpc) for dl in luminosity_distances])
dl_to_z = interp1d(luminosity_distances, redshifts)

In [5]:
for posterior in posteriorsB:
    posterior['redshift'] = dl_to_z(posterior['luminosity_distance'])
    posterior['mass_1'] = posterior['mass_1_det'] / (1 + posterior['redshift'])
    posterior['mass_2'] = posterior['mass_2_det'] / (1 + posterior['redshift'])
    posterior['mass_ratio'] = posterior['mass_2'] / posterior['mass_1']

### Sampling effective spins ($\chi_{eff}$) by parameter estimation

$$\chi_{eff} = \frac{a_1 cost_1 + a_2 q cost_2}{1+q}$$

For each gravitational wave we obtain a set of discrete samples p($\chi_{eff}$|data) (given by `posterior['chieff']` here)

Effective spins are drawn from a simple truncated Gaussian.

<img src="chieff.png">


In [2]:
def chieff(a1, a2, m1, m2, cost1, cost2):
    chieff = (a1*m1*cost1 + a2*m2*cost2)/(m1+m2)
    return chieff

In [7]:
for posterior in posteriorsB:
    posterior['chieff'] = chieff(posterior['a_1'], posterior['a_2'], 
                                 posterior['mass_1'], posterior['mass_2'], 
                                 posterior['cos_tilt_1'], posterior['cos_tilt_2'])

<font color = 'green'>
    <h3>I have a doubt here. Should I be using this LALprior already given with the GWTC-1 catalogue, or should I simulate a new prior?</h3>
    
</font>

Here, simulating the LALprior:

In [3]:
# def conv(parameters):
#     parameters['mass_ratio'] = parameters['mass_2'] / parameters['mass_1']
#     return parameters


simulated_LALprior = bilby.prior.PriorDict(dict(
    mass_1 = Uniform(minimum = 0, maximum = 1, latex_label = 'm1'), 
#     mass_2 = bilby.prior.Cosine('m2'),
    mass_2 = Uniform(minimum = 0, maximum = 1), 
#     mass_ratio = bilby.prior.Constraint(minimum = 0, maximum = 1, latex_label = 'mass_ratio'), 
    a_1 = Uniform(minimum = 0, maximum = 1, latex_label = 'a1'), 
    a_2 = Uniform(minimum = 0, maximum = 1, latex_label = 'a2'), 
    cos_tilt_1 = Uniform(minimum = -1, maximum = 1, latex_label = 'cost1'), 
    cos_tilt_2 = Uniform(minimum = -1, maximum = 1, latex_label = 'cost2')))
#                                            conversion_function = conv)

In [3]:
def conv(parameters):
    parameters['ratio'] = parameters['x'] / parameters['y']
    return parameters

priors = bilby.prior.ConditionalPriorDict(dict(
    x=bilby.prior.Uniform(0, 1, 'x'),
    y=bilby.prior.Cosine('y'),
    ratio=bilby.prior.Constraint(minimum=0, maximum=1, name='ratio')),
    conversion_function=conv)

prior_samples = priors.sample(1000)
prior_samples_array = np.array([prior_samples['x'], prior_samples['y']]).T
corner.corner(prior_samples_array, labels=['x', 'y'])
plt.show()

KeyError: 'ratio'

In [4]:
def convert_x_y_to_z(parameters):
    """
    Function to convert between sampled parameters and constraint parameter.

    Parameters
    ----------
    parameters: dict
        Dictionary containing sampled parameter values, 'x', 'y'.

    Returns
    -------
    dict: Dictionary with constraint parameter 'z' added.
    """
    parameters['z'] = parameters['x'] - parameters['y']
    return parameters

In [9]:
from bilby.core.prior import ConditionalPriorDict, Uniform, Constraint

priors = ConditionalPriorDict(conversion_function=convert_x_y_to_z)
priors['x'] = Uniform(minimum=0, maximum=10)
priors['y'] = Uniform(minimum=0, maximum=10)
priors['z'] = Constraint(minimum=0, maximum=10)

In [10]:
import matplotlib.pyplot as plt

samples = priors.sample(1000000)
plt.hist2d(samples['x'], samples['y'], bins=100, cmap='Blues')
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.tight_layout()
plt.show()
plt.close()

KeyError: 'z'

In [7]:
chi_samples = simulated_LALprior.sample(1000)

# chi_samples

chi_samples_array = np.array([chi_samples['mass_1'], chi_samples['mass_2'], 
                          chi_samples['a_1'], chi_samples['a_2'], 
                          chi_samples['cos_tilt_1'], chi_samples['cos_tilt_2']]).T

# chi_samples_array = np.array(chi_samples)

chi_samples_array

# chi_eff_prior = []

# chi_eff_prior = chieff(chi_samples_array['a_1'], chi_samples_array['a_2'], 
#                  chi_samples_array['mass_1'], chi_samples_array['mass_2'], 
#                  chi_samples_array['cos_tilt_1'], chi_samples_array['cos_tilt_2'])

# corner.corner(chi_samples_array, labels=['a_1', 'a_2', 'mass_1', 'mass_2', 'cos_tilt_1', 'cos_tilt_2'])
# # plt.show()

array({'mass_1': array([0.37459127, 0.42259339, 0.37484628, ..., 0.30105061,
       0.20295791, 0.17913632]), 'mass_2': array([0.58427267, 0.88766025, 0.08202153, ..., 0.72532684,
       0.14631046, 0.31092982]), 'a_1': array([0.53108374, 0.76609201, 0.51433587, ..., 0.63179824,
       0.40936997, 0.5549133 ]), 'a_2': array([0.97360776, 0.84087397, 0.42278715, ..., 0.26772749,
       0.96490872, 0.06524988]), 'cos_tilt_1': array([ 0.31239518,  0.3432398 ,  0.15470395, ..., -0.16719828,
       -0.11684972, -0.34318602]), 'cos_tilt_2': array([-0.57975947, -0.68578883,  0.27760798, ..., -0.70566417,
       -0.98348104,  0.5185064 ])}, dtype=object)

In [30]:
sns.distplot(chi_eff_prior)

NameError: name 'chi_eff_prior' is not defined

In [ ]:
chi_eff_lal_prior = ss.gaussian_kde(chi_eff_prior)

## Hyper-prior models

### $\chi_{\mathrm{eff}}$ 

Effective spins are drawn from a simple truncated Gaussian.

<img src="chieff.png">

### Joint mass and redshift model:

$$p_{astro}(m_1, m_2, z) \propto \frac{(1+z)^{1.7}}{m_1(m_1 - M_{min})} \frac{dV_c}{dz}$$

where $m_1$ and $m_2$ are source frame masses.

As a first step, we assume that the underlying mass distribution does not vary across cosmic time, so that we can factor the joint mass-redshift distribution as:

$$p(m_1, m_2, z) = p(m_1, m_2) p(z)$$

This assumption may break down over a large range of redshifts, as many formation scenarios predict some dependence of the mass distribution on the merger redshift. However, LIGO is only sensitive to redshifts z <~ 1.5, for which this is a good approximation. 

<br>

<font color = 'green'>
I am highly skeptical about my implementation of the mass model. Please suggest corrections. Note that this model does not impose a maximum mass cutoff.
</font>

This is the model given in gwpop.models.redshift:

$$p(z|\gamma, \kappa, z_p) \propto \frac{1}{1 + z}\frac{dV_c}{dz} \psi(z|\gamma, \kappa, z_p)$$

$$\psi(z|\gamma, \kappa, z_p) = \frac{(1 + z)^\gamma}{1 + (\frac{1 + z}{1 + z_p})^\kappa}$$

However, I rewrote its simplified version here, (to account for f(z) as in https://arxiv.org/pdf/1805.10270.pdf (9))